<a href="https://colab.research.google.com/github/hexabenja/DataAnalysisScraper/blob/main/DataWebScr4per.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección


Esta celda instala las bibliotecas necesarias para el web scraping con Langchain, incluidas langchain-community y beautifulsoup4.

In [2]:
%pip install -qU langchain-community beautifulsoup4
%pip install -qU langchain-unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


Esta celda importa bs4 y WebBaseLoader para cargar contenido de una página web específica (En este ejemplo catálogo CREST de sodimac).

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import re
import csv
from datetime import datetime

 Inicializa el loader con la URL y obtiene el documento de forma asíncrona.

In [4]:
page_url = "https://www.sodimac.cl/sodimac-cl/marca/WEBER"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs)

Fetching pages: 100%|##########| 1/1 [00:02<00:00,  2.47s/it]


Esta celda inicializa WebBaseLoader nuevamente, pero esta vez con parámetros adicionales para `bs_kwargs` y `bs_get_text_kwargs` para analizar específicamente solo el contenido de Markdown dentro de la clase `theme-doc-markdown` y para usar separadores y eliminaciones personalizados.

In [5]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(id="testId-searchResults-products", class_="jsx-636341914 search-results--products"),
    },
    bs_get_text_kwargs={"separator": ",", "strip": True},
)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.18it/s]


Esta celda elimina ciertas frases del panel de los resultados con una definición. Al detectar frases tales mencionadas abajo las va a detectar como definición unwanted_phrases.

Limpia el contenido definiendolo como cleaned_content. Usa la definición de cleaned_content para reemplazar por "". Las frases unwanted_phrases de celda superior. Elimina números entre parentesís de puntuacion disponible. Elimina residuos de texto.

In [6]:
#Sintaxis de expresiones regulares regex:
# r'...' = es un raw String por lo cual es texto
# ( ) = es inicio y fin de captura de caracteres
# Backslash es un escape de la funcion especial y debe agregarse como un valor string el valor siguiente (en este caso $)
# \s* = es Cero o mas espacios en blanco
# [ ] = Inicio y final de conjunto de caracteres
# \d = Digito del 1 al 9
# + = Mas conjunto anterior
# ,\d+, = Coma mas uno o mas digitos mas coma
# {2,} = Dos o mas repeticiones de la coma
# ^ = Inicio de lina
# | = Disyuncion OR
# $ = Final de linea
#

def parse_products_simple(content):
    # Lista de frases a eliminar
    unwanted_phrases = [
        "cuotas sin interés",
        "Agregar al Carro",
        "Por Sodimac",
        "Llega mañana",

    ]

    # Limpiar el contenido
    cleaned_content = content

    # Eliminar frases no deseadas
    for phrase in unwanted_phrases:
        cleaned_content = cleaned_content.replace(phrase, "")

    # Eliminar el numero de cuotas)
    cleaned_content = re.sub(r'(\$\s*[\d,\.]+),\d+,', r'\1,', cleaned_content)

    # Eliminar números entre paréntesis como (78), (13), etc y reemplazarlos por nueva fila/linea
    cleaned_content = re.sub(r',?\s*\(\d+\),?', '|NEWLINE|', cleaned_content)

    # Limpiar comas múltiples y espacios extra
    cleaned_content = re.sub(r',{2,}', ',', cleaned_content)
    cleaned_content = re.sub(r',\s*,', ',', cleaned_content)
    cleaned_content = re.sub(r'^\s*,+|,+\s*$', '', cleaned_content)

    # Separar productos por "newline"
    products = []
    parts = cleaned_content.split("|NEWLINE|")

    for part in parts:
        part = part.strip().strip(',').strip()
        if part:
            product_line = "" + part
            products.append(product_line)

    return products

CELDA PARA COMPROBAR SI EL TEXTO PRINTEA CORRECTAMENTE

In [7]:
print(f"{doc.metadata}\n")
print(doc.page_content[:1000].strip())

{'source': 'https://www.sodimac.cl/sodimac-cl/marca/WEBER'}

WEBER,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,Por Sodimac,$  8.590,$  8.332,(270),Agregar al Carro,WEBER,Fragüe Café Claro 1 kg,Por Sodimac,$  1.800,(12),Agregar al Carro,WEBER,Parrilla a Carbón Original Kettle 22" Acero,Por Sodimac,$  116.990,-42%,$  129.990,$  199.990,6,cuotas sin interés,Despacho Gratis,(1380),Agregar al Carro,WEBER,Parrilla a Carbón Original Kettle 18" Acero,Por Sodimac,$  107.990,-10%,$  119.990,6,cuotas sin interés,Armado a $9.990 con CMR,(431),Agregar al Carro,WEBER,Fragüe Gris plata 1 kg,Por Sodimac,$  1.800,(11),Agregar al Carro,WEBER,Cinta fibra vidrio autoadhesiva junturas de tabiquería de fibrocemento,Por Sodimac,$  3.690,(51),Agregar al Carro,WEBER,Autonivelante Superficie Rígido 25 kg,Por Sodimac,$  17.990,(3),Agregar al Carro,WEBER,Imprimante Superficie Rígido 10 kg,Por Sodimac,$  34.990,Agregar al Carro,WEBER,1000cc Pasta adhesiva para cinta de fibra de vidrio,Por Sodimac,$  4

Celda para parsear el texto final delimitado por comas

In [8]:
raw_content = doc.page_content
clean_products = parse_products_simple(raw_content)

final_text = "\n".join(clean_products)
print(final_text)

WEBER,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,$  8.590,$  8.332
WEBER,Fragüe Café Claro 1 kg,$  1.800
WEBER,Parrilla a Carbón Original Kettle 22" Acero,$  116.990,-42%,$  129.990,$  199.990,Despacho Gratis
WEBER,Parrilla a Carbón Original Kettle 18" Acero,$  107.990,-10%,$  119.990,Armado a $9.990 con CMR
WEBER,Fragüe Gris plata 1 kg,$  1.800
WEBER,Cinta fibra vidrio autoadhesiva junturas de tabiquería de fibrocemento,$  3.690
WEBER,Autonivelante Superficie Rígido 25 kg,$  17.990
WEBER,Imprimante Superficie Rígido 10 kg,$  34.990,WEBER,1000cc Pasta adhesiva para cinta de fibra de vidrio,$  4.690
WEBER,Fragüe Gris 5 kg,$  7.200
WEBER,Fragüe Almond 1 kg,$  2.490,Retira mañana
WEBER,6 kilos Masilla para junturas de fibrocemento en pasta.,$  11.990
WEBER,Adhesivo Cerámico Muro Superficie Flexible 25 kg,$  19.290
WEBER,3 litros Tape 1000 pasta adhesiva para junturas,$  10.990
WEBER,Adhesivo Cerámico Piso y muro Superficie Rígido 25 kg,$  5.090,$  4.937
WEBER,Impermeabilizan

In [10]:
# Función simple para exportar a CSV con encabezados
def export_simple_csv(clean_products):
    # Obtener la marca del primer producto para segmentar por marca de productos
    if clean_products:
        first_product = clean_products[0]
        marca = first_product.split(',')[0].strip()  # ← NUEVO: Extrae la marca
    else:
        marca = "productos"  # Por si no hay productos

    # Obtener la fecha actual en día, mes y año
    fecha_actual = datetime.now().strftime("%Y%m%d")

    # Crear nombre del archivo: marca_fecha.csv
    filename = f"{marca}_{fecha_actual}.csv"  # ← CAMBIO: Usa variable marca

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        # Encabezados
        writer.writerow(['Marca', 'Nombre_Producto', 'Precio'])

        # Datos
        for product in clean_products:
            parts = product.split(',')
            marca = parts[0].strip()
            nombre = parts[1].strip() if len(parts) > 1 else ''
            precio = parts[2].strip() if len(parts) > 2 else ''

            writer.writerow([marca, nombre, precio])

    print(f"CSV creado: {filename}")

# Usar la función:
export_simple_csv(clean_products)

CSV creado: WEBER_20250921.csv
